<h1><u>Advancedn topics in ML - Assignment III - task A</u></h1>
<h4>Omer Nivron</h4>
15098443

In [ ]:
import gym
import numpy as np

In [ ]:
#observation=[position of cart, velocity of cart, angle of pole, rotation rate of pole]

In [ ]:
def observationToState(o, thresh):
    s = int(0);
    exp = 1;
    for i in range(len(o)):
        if thresh[i] == 0:
            continue
        ox = (min(thresh[i], max(-thresh[i], o[i])) + thresh[i]);
        val = int((ox * float(steps)) / (2.0 * thresh[i]));
        if val == steps:
            val = steps - 1
        s += val * exp;
        exp *= steps;
    return s;

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
a=env.action_space.sample()

In [ ]:
s = env.reset()

In [ ]:
env.observation_space.shape[0]

In [ ]:
env.step(a)

In [ ]:
env.action_space.n

<h1>part A</h1>

In [ ]:
#create lists to contain total rewards
rList = []

In [ ]:
lr = 1
y = .99
num_episodes = 3
size=(3,5)
return_matrix=np.zeros(size)

In [ ]:
    for i in range(num_episodes):
    #Reset environment and get first new observation
        s = env.reset()
        print(s)
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 300:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.sum(np.random.uniform(0,1)>0.5)
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            r=0
            s = s1
            if d == True:
                r=-1
                rAll += r
                print("Episode finished after {} timesteps".format(j+1))
                break
        rList.append(rAll)
        print 
        print "Score over time: " +  str(sum(rList)/num_episodes)
        return_matrix[i]=np.append(j+1,s)

In [ ]:
return_matrix

<h1>part B</h1>

In [ ]:
#create lists to contain total rewards
rList = []

In [ ]:
lr = 1
y = .99
num_episodes = 100
size=(100,5)
return_matrix=np.zeros(size)

In [ ]:
    for i in range(num_episodes):
    #Reset environment and get first new observation
        s = env.reset()
        print(s)
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 300:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.sum(np.random.uniform(0,1)>0.5)
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            r=0
            s = s1
            if d == True:
                r=-1
                rAll += r
                print("Episode finished after {} timesteps".format(j+1))
                break
        rList.append(rAll)
        print 
        print "Score over time: " +  str(sum(rList)/num_episodes)
        return_matrix[i]=np.append(j+1,s)

In [ ]:
(np.mean(return_matrix,0))

In [ ]:
np.std(return_matrix,0)

<h1>part C</h1>

In [1]:
import gym
import numpy as np
import random
import tensorflow as tf

In [2]:
env = gym.make('CartPole-v0')

[2017-03-24 20:01:35,676] Making new env: CartPole-v0


In [3]:
tf.reset_default_graph()

<h2>Configurations</h2>

In [4]:
epochs=50
lr=[10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 0.5]
lr=lr[2]
batch_s=100
y=.99
n_eps=2000
num_eps_test=10
ran=100

In [5]:
logs_path="/Users/onivron/trained_models/ass_3/tensor_plot/task_A/part_C/part_c_lr_10_3"

<h3>Var creation</h3>
We build data placeholders & variables 

In [6]:
with tf.name_scope('states'):
    states=tf.placeholder("float32",[None,4],name='states')
with tf.name_scope('s_ne'):
    s_ne=tf.placeholder("float32",[None,4],name='s_ne')
with tf.name_scope("weights"):
    W1 = tf.get_variable("W1", [4, 100],initializer=tf.random_normal_initializer(0,0.001))
    W2 = tf.get_variable("W2", [100, 2],initializer=tf.random_normal_initializer(0,0.001))
with tf.name_scope("biases"):
    b1 = tf.get_variable("b1", [100],initializer=tf.constant_initializer(0,0.001))
    b2 = tf.get_variable("b2", [2],initializer=tf.constant_initializer(0,0.001))
with tf.name_scope('targetQ'):
    targetQ=tf.placeholder("float32",[None,2],name='targetQ')

<h3>Define model</h3>

In [7]:
def current_observation_to_action(states):
    h1 = tf.nn.relu(tf.matmul(states,W1)+ b1)
    Q=tf.matmul(h1,W2)+b2
    return Q

In [8]:
def next_observation_to_action(s_ne):
    h1 = tf.nn.relu(tf.matmul(s_ne,W1)+ b1)
    Q_prime=tf.stop_gradient(tf.matmul(h1,W2)+b2)
    return Q_prime

<h3>Define loss & optimizer</h3>

In [9]:
#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
Q=current_observation_to_action(states)
Q_prime=next_observation_to_action(s_ne)

with tf.name_scope('loss'):
    loss=tf.reduce_mean(tf.square((targetQ) -(Q)))
    
with tf.name_scope('train'):   
    trainer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

In [10]:
def collect_offline_observ(num_episodes):
    store=[]
    for i in range(num_episodes):
    #Reset environment and get first new observation
        s = env.reset()
        d = False
        j = 0     
        while j < 300:
            a = np.sum(np.random.uniform(0,1)>0.5)
            s1,r,d,_ = env.step(a)
            r=0
            if (d==True):
                r=-1
                #print("Episode finished after {} timesteps".format(j))            
            tmp_st=np.append(a,s)
            tmp_st=np.append(tmp_st,s1)
            tmp_st=np.append(tmp_st,r)
            store=np.append(store,tmp_st)
            if (d==True):
                break
            j+=1
            s=s1
    store=np.reshape(store,[-1,10])
    return(store)

In [11]:
observations=collect_offline_observ(n_eps)

In [12]:
# Tensorboard summaries for loss & accuracy 
tf.summary.scalar("loss",loss)
summary_op=tf.summary.merge_all()

writer_opt=tf.summary.FileWriter(logs_path,graph=tf.get_default_graph())

In [13]:
saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)

In [14]:
init = tf.global_variables_initializer()

In [15]:
# Launch the graph
sess=tf.Session() 
sess.run(init)

print("------- Reloading previous parameters and continuing training...")
#folder='/Users/onivron/trained_models/ass_3/weights/lr'
saver = tf.train.import_meta_graph(folder+'/accu.meta')
saver.restore(sess, tf.train.latest_checkpoint(folder+'/'))
all_vars = tf.get_collection('vars')
for v in all_vars:
    sess.run(v)

In [31]:
#create lists to contain total rewards and steps per episode
lo_List = []
obser_len=np.shape(observations)[0]
num_batch=int(obser_len/batch_s)
#avg_eps_len=np.zeros(epochs)
for epoch in range(epochs):
    for batch_num in range(num_batch):
        
        batch_rows=np.random.randint(np.shape(observations)[0]-1, size=batch_s)
        
        batch_cur_st=np.reshape(observations[[batch_rows],1:5],[batch_s,4])
        batch_nex_st=np.reshape(observations[[batch_rows],5:9],[batch_s,4])
        
        actions=np.reshape(observations[[batch_rows],0],[100,1])
        
        rewards=np.reshape(observations[[batch_rows+1],-1],[100,1])
        delta=rewards+1
        #delta=1
        # I get for allQ a [1,2] vector 
        current_Q=sess.run(Q,feed_dict={states:batch_cur_st})
        target_Q=np.copy(current_Q)
        
        Q_s_prime=sess.run(Q_prime,feed_dict={s_ne:batch_nex_st})
        maxQ1=np.reshape(np.max(Q_s_prime,1),[100,1])
        
        row_idx = np.array(range(batch_s))
        target_Q[row_idx[:,None],[actions]]=delta*y*maxQ1+rewards 
        _,lo,summary=sess.run([trainer,loss,summary_op],feed_dict={states:batch_cur_st,targetQ:target_Q})
        lo_List.append(lo)
        #if ((epoch%2==0) & (batch_num==100)):
            #eps_len=np.zeros(100)
        #    for eps in range(num_eps_test):
        #        s=env.reset()
        #        s=np.reshape(s,[1,4])
        #        j=0
        #        while j < 300:
        #            j+=1
        #            test_Q=sess.run(Q_prime,feed_dict={states:s})
        #            act=np.argmax(test_Q)
        #            s1,r,d,_ = env.step(act)
        #            s1=np.reshape(s1,[1,4])
        #            s=s1
        #            if (d==True):
                        #in_eps_len=np.copy(j)
                        #eps_len[eps]=in_eps_len
        #                break       
        #    print("Episode finished after {} timesteps".format(j))  
            #avg_eps_len[epoch/2]=np.mean(eps_len)
    print (lo)
    writer_opt.add_summary(summary,(epoch))
    folder='/Users/onivron/trained_models/ass_3/weights/task_A_part_c/'+str(lr)
    save_path=saver.save(sess,folder+'/')

/Users/onivron/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:27: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future


0.00484276
0.00192754
0.00472981
0.00559159
0.00461164
0.00272932
0.00536262
0.00266067
0.00435515


KeyboardInterrupt: 

In [18]:
eps_len=np.zeros(100)
avg_eps_len=np.zeros(epochs)
for eps in range(100):
    #Reset environment and get first new observation
    s = env.reset()
    s_=np.reshape(s,[1,4])
    d = False
    j = 0     
    while j < 300:
        a = sess.run(Q_prime,feed_dict={s_ne:s_})
        ac=np.argmax(a)
        s1,r,d,_ = env.step(ac)
        r=0
        if (d==True):
            r=-1
            print("Episode finished after {} timesteps".format(j))
            in_eps_len=np.copy(j)
            eps_len[eps]=in_eps_len
            break
        j+=1
        s_=np.reshape(s1,[1,4])
avg_eps_len=np.mean(eps_len)

Episode finished after 7 timesteps
Episode finished after 9 timesteps
Episode finished after 10 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode finished after 9 timesteps
Episode finished after 8 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode finished after 7 timesteps
Episode finished after 7 timesteps
Episode finished after 9 timesteps
Episode finished after 8 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode finished after 8 timesteps
Episode finished after 7 timesteps
Episode finished after 9 timesteps
Episode finished after 9 timesteps
Episode finished after 8 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode finished after 7 timesteps
Episode finished after 8 timesteps
Episode finished after 7 timesteps
Episode finished after 9 timesteps
Episode finished after 9 timesteps
Episode finished after 7 timesteps
Episode finished af

In [ ]:
avg_eps_len

<h1>part D</h1>

In [ ]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
lr=[10^-5, 10^-4, 10^-3, 10^-2, 10^-1, 0.5]

In [ ]:
tf.reset_default_graph()

In [ ]:
#Initialize table with all zeros
#Q = np.zeros([env.observation_space.shape[0],env.action_space.n])
# Set learning parameters
#lr = 1
#y = .99
#num_episodes = 2

In [ ]:
#create lists to contain total rewards
#rList = []

In [ ]:
#    for i in range(num_episodes):
#    #Reset environment and get first new observation
#        s = env.reset()
#        print(s)
#        rAll = 0
#        d = False
#        j = 0
#        #The Q-Table learning algorithm
#        while j < 99:
#            j+=1
#            #Choose an action by greedily (with noise) picking from Q table
#            a = np.sum(np.random.uniform(0,1)>0.5)
 ##           #Get new state and reward from environment
#           s1,r,d,_ = env.step(a)
#            r=0
            #Update Q-Table with new knowledge
#            Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
#            s = s1
#            if d == True:
#                r=-1
#                rAll += r
 #               print("Episode finished after {} timesteps".format(j+1))
#               break
#        rList.append(rAll)
#        print "Score over time: " +  str(sum(rList)/num_episodes)
 #       print "Final Q-Table Values"
#       print Q

Initialize network with random small weights
Observe starting state s
Repeat:
Choose a from s using policy derived from Q (e.g. greedy) Take action a, observe r, s′
Update Q by backpropagating the error signal given by E = Q(s, a) − r + γ maxa Q(s′, a)
s ← s′

In [ ]:
state_dim   = env.observation_space.shape[0]
num_actions = env.action_space.n
batch_s=1

In [ ]:
states=tf.placeholder(shape=[batch_s,4],dtype=tf.float32) 
W1 = tf.get_variable("W1", [4, 100],
                       initializer=tf.random_normal_initializer())
b1 = tf.get_variable("b1", [100],
                       initializer=tf.constant_initializer(0))

In [ ]:
W2 = tf.get_variable("W2", [100, 2],initializer=tf.random_normal_initializer())
b2 = tf.get_variable("b2", [2],initializer=tf.constant_initializer(0))
nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)

In [ ]:
def observation_to_action(states):
  # define policy neural network

  h1 = tf.nn.relu(tf.matmul(states, W1) + b1)

  Q=tf.matmul(h1, W2) + b2
  predict=tf.argmax(Q,1)
  return predict,Q



In [ ]:
#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
pred,Q=observation_to_action(states)
loss = tf.reduce_sum(tf.square(tf.stop_gradient(nextQ) - Q))
trainer = tf.train.GradientDescentOptimizer(learning_rate=10**(-5))
updateModel = trainer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
y = .99
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
sess=tf.Session()
sess.run(init)
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    s=np.reshape(s,[1,4])
    rAll = 0
    d = False
    j = 0
    #The Q-Network
    while j < 300:
        #env.render()
        j+=1
        a = np.sum(np.random.uniform(0,1)>0.5)
        # I get for allQ a [1,2] vector 
        allQ = sess.run([Q],feed_dict={states:s})
        s1,r,d,_ = env.step(a)
        s1=np.reshape(s1,[1,4])
        r=0
        Q1 = sess.run(Q,feed_dict={states:s1})            
        maxQ1 = np.max(Q1)
        targetQ = allQ
        (targetQ)[0][0,a] =(y*(maxQ1)+r)      
        targetQ=np.reshape(targetQ,[1,2])
        _,lo=sess.run([updateModel,loss],feed_dict={states:s1,nextQ:targetQ})
        rAll += r
        s = s1
        if d == True:
            r=-1
            rAll += r
            if (i%100==0):
                print("Episode finished after {} timesteps with loss {}".format(j+1,lo))            
            break
    jList.append(j)

In [ ]:
## testing



In [ ]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

state_dim   = env.observation_space.shape[0]
num_actions = env.action_space.n

def observation_to_action(states):
  # define policy neural network
  W1 = tf.get_variable("W1", [state_dim, 100],
                       initializer=tf.random_normal_initializer())
  b1 = tf.get_variable("b1", [100],
                       initializer=tf.constant_initializer(0))
  h1 = tf.nn.relu(tf.matmul(states, W1) + b1)
  W2 = tf.get_variable("W2", [100, num_actions],
                       initializer=tf.random_normal_initializer())
  b2 = tf.get_variable("b2", [num_actions],
                       initializer=tf.constant_initializer(0))
  Q = tf.matmul(h1, W2) + b2
  predict=tf.argmax(q,1)
  return predict,Q




#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Q))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

init = tf.initialize_all_variables()

# Set learning parameters
y = .99
num_episodes = 1
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Network
        while j < 300:
            j+=1
            a = np.sum(np.random.uniform(0,1)>0.5)
            allQ = sess.run([Q],feed_dict={inputs1:np.identity(16)[s:s+1]})
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            r=0
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Q,feed_dict={inputs1:s1})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:s1,nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                r=-1
                rAll += r
                print("Episode finished after {} timesteps".format(j+1))
                break
        jList.append(j)
        rList.append(rAll)
print "Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%"

In [ ]:
10**(-5)

In [ ]:
batch_s=100

In [ ]:
(int(600000/batch_s))

In [ ]:
34%2

In [ ]:
in_eps_len=1

In [ ]:
in_eps_len+=1

In [ ]:
in_eps_len